In [3]:
import numpy as np
import pandas as pd
import itertools
from __future__ import division
from sklearn.tree import tree, DecisionTreeClassifier, export_graphviz
from sklearn import cluster

%matplotlib inline
pd.set_option("display.max_columns", 500)
pd.set_option("max_rows", 1000)

In [5]:
filePath = 'datasets/NYPD_Motor_Vehicle_Collisions_weather4.csv'
colls = pd.read_csv(filePath)
# colls = colls[(colls['VEHICLE TYPE CODE 1'] == 'TAXI') | \
#              (colls['VEHICLE TYPE CODE 2'] == 'TAXI') | \
#              (colls['VEHICLE TYPE CODE 3'] == 'TAXI') | \
#              (colls['VEHICLE TYPE CODE 4'] == 'TAXI') | \
#              (colls['VEHICLE TYPE CODE 5'] == 'TAXI')]

colls = colls[colls['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']
colls = colls[pd.notnull(colls['LOCATION'])]

colls['HOUR'] = colls.TIME.str.split(':').str.get(0)
colls['MONTH'] = colls.DATE.str.split('/').str.get(0)

colls = colls[pd.notnull(colls['HOUR'])]

colls.head()

,Unnamed: 0,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,NUMBER OF PERSONS KILLED,NUMBER OF PEDESTRIANS INJURED,NUMBER OF PEDESTRIANS KILLED,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED,NUMBER OF MOTORIST INJURED,NUMBER OF MOTORIST KILLED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,UNIQUE KEY,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,Conditions,Precipitationmm,TemperatureC,VisibilityKm,HOUR,MONTH
4,4,03/13/2016,23:00,BROOKLYN,11206,40.706653,-73.950406,"(40.7066527, -73.9504063)",UNION AVENUE,MONTROSE AVENUE,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,3405121,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,03
6,6,03/13/2016,9:48,BROOKLYN,11212,40.661997,-73.919593,"(40.661997, -73.9195931)",KINGS HIGHWAY,EAST 98 STREET,NaN,0,0,0,0,0,0,0,0,Passenger Distraction,Unspecified,NaN,NaN,NaN,3404744,PASSENGER VEHICLE,UNKNOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,03
9,9,03/13/2016,9:46,QUEENS,11106,40.756580,-73.929752,"(40.75658, -73.9297516)",36 AVENUE,31 STREET,NaN,0,0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Unspecified,NaN,NaN,NaN,3404995,LIVERY VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,03
11,11,03/13/2016,16:00,QUEENS,11413,40.672174,-73.761021,"(40.6721741, -73.7610205)",BEDELL STREET,140 AVENUE,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,NaN,NaN,NaN,3404948,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,03
14,14,03/13/2016,15:20,MANHATTAN,10019,40.761719,-73.982807,"(40.7617191, -73.9828066)",WEST 51 STREET,7 AVENUE,NaN,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,3404581,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,03


In [152]:
len(colls)

267843

In [6]:
def encode_column(df, target_column):
#     print "copying dataset"
    df_mod = df.copy()
#     print "finding uniques"
    targets = pd.Series(df_mod[target_column].unique())
#     print "mapping to ints"
    map_to_int = {name: n for n, name in enumerate(targets)}
#     print "replacing in data set"
    df_mod[target_column+"_encoded"] = df_mod[target_column].replace(map_to_int)
#     df_mod[target_column+"_encoded"] = df_mod[target_column].replace(targets[targets == target_column])
    return (df_mod, targets)

def train_tree(prediction, features, dataset):
    clf = tree.DecisionTreeClassifier()
    print "TRAINING WITH %d SAMPLES" % len(dataset) 
    X = np.array(dataset[features])
    Y = np.array(list(itertools.chain(*dataset[[prediction]].values)))
    return clf.fit(X, Y)

def test_tree(clf, test_data, features):
    return clf.predict(test_data[features])

def convert_encoded_district_to_str(preditions):
    return map(lambda p: districts[p], preditions)

def test_prediction(target_label, clf, test_data, features, encoded_map):
    corrects = 0
    predictions = test_tree(clf, test_data[features], features)
    for i in range(0, len(predictions)):
        if predictions[i] == test_data.iloc[i][target_label]:
            corrects += 1
    print "FOUND %d CORRECT PREDICTIONS" % corrects
    return corrects / len(predictions)

def knn(k, dataset, colums):
    md = cluster.KMeans(n_clusters=k).fit(dataset[colums])
    return md.predict(dataset[colums]),md.cluster_centers_

In [31]:
target_label = 'ZIP CODE'

data = colls[pd.notnull(colls[target_label])]

# Encoding target/label column
# mdata, target = encode_column(data, 'ZIP CODE')
mdata, target = encode_column(data, target_label)

# Encode the feature columns
mdata, _ = encode_column(mdata, 'BOROUGH')
# mdata, _ = encode_column(mdata, 'CONTRIBUTING FACTOR VEHICLE 1')
# mdata, _ = encode_column(mdata, 'VEHICLE TYPE CODE 1')
mdata, _ = encode_column(mdata, 'Conditions')

# KNN
s, cen = knn(10, mdata, ['LONGITUDE', 'LATITUDE'])
mdata['KMEANS'] = s


# Splitting date and time into month and hour
# mdata['HOUR'] = data.TIME.str.split(':').str.get(0)
# mdata['MONTH'] = data.DATE.str.split('/').str.get(1)

In [32]:
# Features for prediction
features = ['KMEANS', 'Conditions_encoded']

# Split data set into training and test data
train_data = mdata.head(int(mdata.BOROUGH.count() * 0.90))
test_data = mdata.tail(int(mdata.BOROUGH.count() * 0.10))

target_label_encoded = target_label+'_encoded'

# Train the decision tree
clf = train_tree(target_label_encoded, features, train_data)

# Test the decision tree
print "Prediction accuracy %f" % test_prediction(target_label_encoded, clf, test_data, features, target)

TRAINING WITH 241015 SAMPLES
FOUND 3554 CORRECT PREDICTIONS
Prediction accuracy 0.132716


In [168]:
nan = colls[pd.isnull(colls.TemperatureC)]
print len(nan)
print len(colls)

17546
267843


# Decision Tree Log
## Finding ZIP CODE with HOUR, MONTH, BOROUGH
- TRAINING WITH 438514 SAMPLES
- FOUND 7884 CORRECT PREDICTIONS
- Prediction accuracy 0.053937
## Finding ZIP CODE with HOUR, MONTH, BOROUGH, CFV1, VTC1
- TRAINING WITH 526217 SAMPLES
- FOUND 3243 CORRECT PREDICTIONS
- Prediction accuracy 0.055466
## Finding VTC1 with HOUR, MONTH, BOROUGH, CFV1
- TRAINING WITH 438514 SAMPLES
- FOUND 77135 CORRECT PREDICTIONS
- Prediction accuracy 0.527704

In [ ]:
colls[colls['DATE'] == "03/08/2015"].head(1000).sort_values(by='TIME')